In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from dataclasses import dataclass, field
from typing import List, Tuple

%matplotlib ipympl

In [3]:
x, y, h = 10, 20, 30

space = np.array([
    [1, 2, 3, 1],
    [1, 2, 3, 1],
])

In [9]:
@dataclass
class Camera:
    focal_length: float
    resolution: Tuple[int, int]

    position: np.ndarray#=field(default_factory=np.array([0., 0., 0.]))
    angle: np.ndarray#=field(default_factory=np.array([0., 1., 0.]))  # angle in degrees

    def __post_init__(self):
        psi, theta, phi = self.angle  # R_x, R_y, R_z
        psi, theta, phi = psi, theta, phi

        sin_psi, cos_psi = np.sin(np.deg2rad(psi)), np.cos(np.deg2rad(psi))
        sin_theta, cos_theta = np.sin(np.deg2rad(theta)), np.cos(np.deg2rad(theta))
        # sin_phi, cos_phi = np.sin(np.deg2rad(phi)), np.cos(np.deg2rad(phi))

        v = np.array([cos_theta*cos_psi, sin_theta*cos_psi, sin_psi])
        self.unit_angle = v / np.linalg.norm(v)

        sin_psi, cos_psi = np.sin(np.deg2rad(psi)), np.cos(np.deg2rad(psi))
        sin_theta, cos_theta = np.sin(np.deg2rad(theta)), np.cos(np.deg2rad(theta))
        sin_phi, cos_phi = np.sin(np.deg2rad(phi)), np.cos(np.deg2rad(phi))

        # rotation = np.array([
        #     [cos_theta*cos_phi, sin_psi*sin_theta*cos_phi - cos_psi*sin_phi, cos_psi*sin_theta*cos_phi + sin_psi*sin_phi],
        #     [cos_theta*sin_phi, sin_psi*sin_theta*sin_phi + cos_psi*cos_theta, cos_psi*sin_theta*sin_phi - sin_psi*sin_theta],
        #     [-sin_theta, sin_phi*cos_theta, cos_phi*cos_theta]
        # ])
        rotation = np.array([
            [cos_theta*cos_psi, sin_psi*sin_theta*cos_phi - cos_psi*sin_phi, cos_psi*sin_theta*cos_phi + sin_psi*sin_phi],
            [cos_theta*sin_psi, sin_psi*sin_theta*sin_phi + cos_psi*cos_theta, cos_psi*sin_theta*sin_phi - sin_psi*sin_theta],
            [sin_psi, sin_phi*cos_theta, cos_phi*cos_theta]
        ])
        unit_vec = np.array([1, 0, 0])
        print(unit_vec.shape, rotation.shape)
        v = np.matmul(rotation, unit_vec.T)
        print(self.unit_angle, v)
        self.unit_angle = v / np.linalg.norm(v)

    def _rotation_angle(self, std):
        cos = np.dot(std, self.angle)  # / length of angle, which is 1 as it is a unit vector
        sin = np.linalg.norm(np.cross(std, self.angle))

        return cos, sin

    '''
        Transform world coordinate to pixel coordinate
            - World to camera coordinate
            - Camera to pixel coordinate
    '''
    def transform(self, 
        space,
        world_position
    ):
        # convention assumption on viewing angles
        translation = np.array(self.position - world_position)  # row vector

        psi, theta, phi = self.angle  # R_x, R_y, R_z
        sin_psi, cos_psi = np.sin(np.deg2rad(psi)), np.cos(np.deg2rad(psi))
        sin_theta, cos_theta = np.sin(np.deg2rad(theta)), np.cos(np.deg2rad(theta))
        sin_phi, cos_phi = np.sin(np.deg2rad(phi)), np.cos(np.deg2rad(phi))

        rotation = np.array([
            [cos_theta*cos_phi, sin_psi*sin_theta*cos_phi - cos_psi*sin_phi, cos_psi*sin_theta*cos_phi + sin_psi*sin_phi],
            [cos_theta*sin_phi, sin_psi*sin_theta*sin_phi + cos_psi*cos_theta, cos_psi*sin_theta*sin_phi - sin_psi*sin_theta],
            [-sin_theta, sin_phi*cos_theta, cos_phi*cos_theta]
        ])
        m = np.zeros((4, 4))
        m[0:3, 0:3] = rotation
        m[0:3, -1] = translation
        m[-1, -1] = 1

        intrinsic = np.array([
            [self.focal_length, 0, 0, 0],
            [0, self.focal_length, 0, 0],
            [0, 0, 1, 0]
        ])

        abc = np.array([
            [1/self.resolution[0], 0, 1/self.resolution[0]/2],
            [0, 1/self.resolution[1], 1/self.resolution[1]/2],
            [0, 0, 1]
        ])

        # image_space = np.matmul(space, m)
        image_space = np.matmul(m, space.T).T
        camera_space = np.matmul(intrinsic, image_space.T).T
        pixel_space = np.matmul(abc, camera_space.T).T

        return pixel_space

    # def pixel2ray(self, pixel):
    #     pixel += np.array([.5, .5, 0])
    #     # intrinsic = np.array([
    #     #     [self.focal_length, 0, self.resolution[0]//2],
    #     #     [0, self.focal_length, self.resolution[1]//2],
    #     #     [0, 0, 1]
    #     # ])
        
    #     # psi, theta, phi = self.angle  # R_x, R_y, R_z
    #     # sin_psi, cos_psi = np.sin(np.deg2rad(psi)), np.cos(np.deg2rad(psi))
    #     # sin_theta, cos_theta = np.sin(np.deg2rad(theta)), np.cos(np.deg2rad(theta))
    #     # sin_phi, cos_phi = np.sin(np.deg2rad(phi)), np.cos(np.deg2rad(phi))

    #     # rotation = np.array([
    #     #     [cos_theta*cos_phi, sin_psi*sin_theta*cos_phi - cos_psi*sin_phi, cos_psi*sin_theta*cos_phi + sin_psi*sin_phi],
    #     #     [cos_theta*sin_phi, sin_psi*sin_theta*sin_phi + cos_psi*cos_theta, cos_psi*sin_theta*sin_phi - sin_psi*sin_theta],
    #     #     [-sin_theta, sin_phi*cos_theta, cos_phi*cos_theta]
    #     # ])

    #     # norm_pixel = np.matmul(np.linalg.inv(intrinsic), pixel.T).T
    #     # world_ray = np.matmul(np.linalg.inv(rotation), norm_pixel.T).T
    #     return world_ray

camera = Camera(16.16003, (1536, 1536), np.array([8, 2.5, 3]), (17.288, 232.093, 0))
# pixel_coordinate = camera.transform(space, np.array([0, 0, 0]))
# print(pixel_coordinate[0])

# pixel = np.array([
#     [100, 200, 1],
#     [300, 100, 1],
# ])
# print(pixel.shape)
# rays = camera.pixel2ray(pixel)
# print(rays.shape)

(3,) (3, 3)
[-0.58662572 -0.75336402  0.2971749 ] [-0.6143816   0.          0.78900903]


In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
# ax.view_init(elev=0, azim=90, roll=0)  # switch y- and z-axis
ax.set_xlabel("X")
ax.set_ylabel("Z")
ax.set_zlabel("Y")

ax.scatter(0, 0, 0)  # origin
ax.scatter(*camera.position)
ax.quiver(*camera.position, *camera.unit_angle, color='blue')
# ax.quiver(*camera.position, *v, color='red')

# plt.axis('off')
plt.show()